In [1]:
import numpy as np 
import torch
import torch.utils.data as Data
import torch.optim as optim

from MyNet import Net, MetaDataSet, MyLoss1, MyLoss

# hyper parameters
BATCH_SIZE = 10
EPOCH_SIZE = 10

LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY =  0.0005

DATASET_SHUFFLE = False
num_WORKERS = 2

TASK = 'regression'

XI = 0.0001


save_path = './'
metadata_dir = 'E:/metadata数据集/new_bigmetadata/australian/query_time/'

# net = Net(n_feature=396, first_n_hidden=100, second_n_hidden=20, n_output=2)
net_bn = Net(n_feature=836, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=True)
net_no_bn = Net(n_feature=836, first_n_hidden=200, second_n_hidden=100, n_output=1, batch_normalization=False)

nets = [net_bn, net_no_bn]
optimizers = [optim.Adam(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY) for net in nets]
criterion = MyLoss1(XI)
print(nets)
print(optimizers)
print(criterion)

[Net(
  (input_bn): BatchNorm1d(836, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=836, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features=100, bias=True)
  (second_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
), Net(
  (input_bn): BatchNorm1d(836, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_hidden): Linear(in_features=836, out_features=200, bias=True)
  (first_bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (second_hidden): Linear(in_features=200, out_features=100, bias=True)
  (second_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
)]
[Adam

In [2]:
# print('Preparing the dataset....')
# loader = MetaDataSet(metadata_dir, BATCH_SIZE, num_WORKERS, TASK)
# print("Dataset is ready!")

metadata = np.load('E:/metadata数据集/new_bigmetadata/australian/query_time/2australian30_big_metadata30.npy')

# X = metadata[:, 0:396]
# y = metadata[:, 396]
X = metadata[:, 0:418]
y = metadata[:, 418]
# y[np.where(y>0)[0]] = 1
# y[np.where(y<=0)[0]] = 0

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)
X_tensor = X_tensor.float()
y_tensor = y_tensor.float()
print(torch.max(y_tensor))
print(torch.min(y_tensor))
print(torch.mean(y_tensor))

print(torch.min(torch.abs(y_tensor)))

print(X_tensor.dtype)
print(y_tensor.dtype)
# troch_dataset = Data.TensorDataset(data_tensor=X_tensor, target_tensor=y_tensor)
troch_dataset = Data.TensorDataset(X_tensor, y_tensor)

loader = Data.DataLoader(dataset=troch_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=DATASET_SHUFFLE,
                        num_workers=num_WORKERS)

tensor(0.3285)
tensor(-0.2587)
tensor(0.0152)
tensor(0.)
torch.float32
torch.float32


In [3]:
# tarining the NN
for epoch in range(EPOCH_SIZE):
    running_loss0 = 0.0
    running_loss1 = 0.0
    for step, (batch_x, batch_y) in enumerate(loader):
#         print('the shape of batch_X', batch_x.size())
#         print('the shape of batch_y', batch_y.size())
#         print(batch_x)
        min_b = int(EPOCH_SIZE/2)
        pari_X = torch.cat((batch_x[0:min_b,:],batch_x[min_b:,:]), 1)
        pari_y = batch_y[0:min_b] - batch_y[min_b:]
        for net, optimizer in zip(nets, optimizers):
            optimizer.zero_grad()
            outputs = net(pari_X)
            loss = criterion(outputs, pari_y)
#             print('the dtype of loss ', loss.dtype)
#             print('the shape of loss', loss.size())
#             print('loss ', loss)
            loss.backward()
            optimizer.step()
            
            if net == nets[0]:
                i = 0
                # print statistics
                running_loss0 += loss.item()
                # print every 2000 mini-batches
                if step % 1000 == 999:    
                    print('net',i,' [%d, %5d] loss: %.3f' %
                        (epoch + 1, step + 1, running_loss0 / 2000))
                    running_loss0 = 0.0
            else:
                i = 1
                # print statistics
                running_loss1 += loss.item()
                # print every 2000 mini-batches
                if step % 1000 == 999:    
                    print('net',i,' [%d, %5d] loss: %.3f' %
                        (epoch + 1, step + 1, running_loss1 / 2000))
                    running_loss1 = 0.0

net 0  [1,  1000] loss: 0.212
net 1  [1,  1000] loss: 0.895
net 0  [1,  2000] loss: 0.054
net 1  [1,  2000] loss: 0.015
net 0  [2,  1000] loss: 0.022
net 1  [2,  1000] loss: 0.003


KeyboardInterrupt: 

In [ ]:
# save the parameters in NN
torch.save({
            'model_state_dict': nets[0].state_dict(),
            'optimizer_state_dict': optimizers[0].state_dict(),
            }, './regression_bn_net_parameters.pth')
torch.save({
            'model_state_dict': nets[1].state_dict(),
            'optimizer_state_dict': optimizers[1].state_dict(),
            }, './regression_nobn_net_parameters.pth')

In [13]:
op = optimizers[0]

for x in op.state_dict().keys():
    print(x)
print(op.state_dict()['param_groups'][0]['lr'])
pg = op.state_dict()['param_groups']
print(pg[0]['lr'])

state
param_groups
0.001
0.001
